In [1]:
!python -m pip install elasticsearch
!wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
!tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
!chown -R daemon:daemon elasticsearch-7.9.2



In [1]:
import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.9.2/bin/elasticsearch'],
                   stdout=PIPE, stderr=STDOUT,
                   preexec_fn=lambda: os.setuid(1)  # as daemon
                  )
# wait until ES has started
! sleep 30

In [5]:
! /opt/ml/elasticsearch-7.9.2/bin/elasticsearch-plugin install analysis-nori

-> Installing analysis-nori
-> Downloading analysis-nori from elastic
[=================================================] 100%   
-> Installed analysis-nori


In [41]:
es_server.kill()

In [80]:
import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.9.2/bin/elasticsearch'],
                   stdout=PIPE, stderr=STDOUT,
                   preexec_fn=lambda: os.setuid(1)  # as daemon
                  )
# wait until ES has started
! sleep 30

In [2]:
from elasticsearch import Elasticsearch
es = Elasticsearch('localhost:9200')

In [3]:
es.info()

{'name': 'da4bf02f6143',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'X61iJpvvQOyzcAbooki-Ag',
 'version': {'number': '7.9.2',
  'build_flavor': 'default',
  'build_type': 'tar',
  'build_hash': 'd34da0ea4a966c4e49417f2da2f244e3e97b4e6e',
  'build_date': '2020-09-23T00:45:33.626720Z',
  'build_snapshot': False,
  'lucene_version': '8.6.2',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [4]:
es.indices.get('document')

{'document': {'aliases': {},
  'mappings': {'properties': {'text': {'type': 'text',
     'analyzer': 'my_analyzer',
     'similarity': 'my_similarity'},
    'title': {'type': 'text',
     'analyzer': 'my_analyzer',
     'similarity': 'my_similarity'}}},
  'settings': {'index': {'number_of_shards': '1',
    'provided_name': 'document',
    'similarity': {'my_similarity': {'type': 'BM25'}},
    'creation_date': '1621328153621',
    'analysis': {'filter': {'my_shingle_f': {'type': 'shingle'}},
     'analyzer': {'my_analyzer': {'filter': ['lowercase',
        'my_shingle_f',
        'nori_readingform',
        'nori_number'],
       'decompound_mode': 'mixed',
       'type': 'custom',
       'stopwords': '_korean_',
       'tokenizer': 'nori_tokenizer'}}},
    'number_of_replicas': '1',
    'uuid': 'q6EyPexJQgqqrpxKlarvEA',
    'version': {'created': '7090299'}}}}}

In [5]:
import json
import pandas as pd

with open('/opt/ml/code/data/wikipedia_documents.json', 'r') as f:
    wiki_data = pd.DataFrame(json.load(f)).transpose()

In [5]:
es.indices.delete(index="document", ignore=[400, 404])

05/20/2021 02:31:26 - INFO - elasticsearch -   DELETE http://localhost:9200/document [status:200 request:0.153s]


{'acknowledged': True}

In [ ]:
from tqdm import tqdm
mapping = {
                      'settings':{
                          'analysis':{
                              'analyzer':{
                                  'my_analyzer':{
                                      "type": "custom",
                                      'tokenizer':'nori_tokenizer',
                                      'decompound_mode':'mixed',
                                      'stopwords':'_korean_',
                                      "filter": ["lowercase",
                                                 "my_shingle_f",
                                                 "nori_readingform",
                                                 "nori_number"]
                                  }
                              },
                              'filter':{
                                  'my_shingle_f':{
                                      "type": "shingle"
                                  }
                              }
                          },
                          'similarity':{
                              'my_similarity':{
                                  'type':'BM25',
                              }
                          }
                      },
                      'mappings':{
                          'properties':{
                              'title':{
                                  'type':'text',
                                  'analyzer':'my_analyzer',
                                  'similarity':'my_similarity'
                              },
                              'text':{
                                  'type':'text',
                                  'analyzer':'my_analyzer',
                                  'similarity':'my_similarity'
                              }
                          }
                      }
                  }

from haystack.document_store import ElasticsearchDocumentStore

# from haystack.document_store.elasticsearch import ElasticsearchDocumentStore
document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document", custom_mapping=mapping)

import json
with open('/opt/ml/code/data/wikipedia_documents.json', "r") as f:
    wiki = json.load(f)
contexts = list(dict.fromkeys([v['text'] for v in wiki.values()]))


dicts = [
    {
        'text': context,
        'meta': {}
    } for context in tqdm(contexts)
]
document_store.write_documents(dicts)

05/20/2021 02:31:27 - INFO - elasticsearch -   HEAD http://localhost:9200/ [status:200 request:0.006s]
05/20/2021 02:31:27 - INFO - elasticsearch -   PUT http://localhost:9200/document [status:200 request:0.194s]
05/20/2021 02:31:27 - INFO - elasticsearch -   HEAD http://localhost:9200/label [status:200 request:0.012s]
100%|██████████| 56737/56737 [00:00<00:00, 700431.27it/s]
05/20/2021 02:31:31 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:2.765s]
05/20/2021 02:31:34 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:2.401s]
05/20/2021 02:31:36 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:2.279s]
05/20/2021 02:31:37 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.227s]
05/20/2021 02:31:39 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.18

In [ ]:
#only retriever
from haystack.retriever import ElasticsearchRetriever
retriever = ElasticsearchRetriever(document_store)
from haystack.pipeline import DocumentSearchPipeline
pipe = DocumentSearchPipeline(retriever)

In [70]:
# #reader(FARM of Transformer) +retriever
# from haystack.reader.farm import FARMReader
# from haystack.reader.transformers import TransformersReader
# reader = TransformersReader(model_name_or_path="/opt/ml/code/models/koelectra_test",max_seq_len=300, doc_stride=120)

# from haystack.pipeline import ExtractiveQAPipeline
# pipe = ExtractiveQAPipelinee(reader,retriever)

In [ ]:
from datasets import load_from_disk
testset=load_from_disk('/opt/ml/code/data/test_dataset')
testset=testset['validation']

In [59]:
#dictionary로 top1 문서내용을 json으로 저장
result={}
#for i in range(10): #len(testset['question'])
for example in testset:
    question=example["question"]
    prediction = pipe.run(query=question, top_k_retriever=10, top_k_reader=2)
    from haystack.utils import print_answers
    if prediction['answers'][0]['probability']>0.99:
        result[example["id"]]=prediction['answers'][0]['answer']
    else:
        result[example["id"]]=None

with open('elastic_reader.json', "w") as writer:
    writer.write(json.dumps(result, indent=4, ensure_ascii=False) + "\n")

05/19/2021 22:01:52 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.014s]
05/19/2021 22:01:53 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.027s]
05/19/2021 22:01:54 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.023s]
05/19/2021 22:01:54 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.017s]
05/19/2021 22:01:55 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.014s]
05/19/2021 22:01:55 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.020s]
05/19/2021 22:01:56 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.012s]
05/19/2021 22:01:56 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.016s]
05/19/2021 22:01:57 - INFO - elasticsearch -   P

In [ ]:
#top1 score json으로 저장
save_score={}

for idx, example in enumerate(tqdm(testset, desc="elasticsearch: ")):
    # relev_doc_ids = [el for i, el in enumerate(self.ids) if i in doc_indices[idx]]
    question=example["question"]
    top_k_docs = pipe.run(question, top_k_retriever=30)

    query = {
        'query':{
            'bool':{
                'must':[
                          {'match':{'text':question}}
                ],
                'should':[
                          {'match':{'text':question}}
                ]
            }
        }
    }
    doc = es.search(index='document',body=query,size=30)['hits']['hits']

    
    save_score[example['id']]=doc[0]['_score']
    
with open('top1_score.json', "w") as writer:
    writer.write(json.dumps(save_score, indent=4, ensure_ascii=False) + "\n")

In [34]:
#datafram을 pickle로 top5 저장
import pandas as pd
total = []

for idx, example in enumerate(tqdm(testset, desc="elasticsearch: ")):
    # relev_doc_ids = [el for i, el in enumerate(self.ids) if i in doc_indices[idx]]
    question=example["question"]
    top_k_docs = pipe.run(question, top_k_retriever=30)

    query = {
        'query':{
            'bool':{
                'must':[
                          {'match':{'text':question}}
                ],
                'should':[
                          {'match':{'text':question}}
                ]
            }
        }
    }
    doc = es.search(index='document',body=query,size=30)['hits']['hits']

    
    tmp = {
        "question": example["question"],
        "id": example['id'],
        "context_id": doc[0]['_id'],  # retrieved id
        "context": doc[0]['_source']['text']+doc[1]['_source']['text']+doc[2]['_source']['text']+doc[3]['_source']['text']+doc[4]['_source']['text']  # retrieved doument
    }
 
    if 'context' in example.keys() and 'answers' in example.keys():
        tmp["original_context"] = example['context']  # original document
        tmp["answers"] = example['answers']           # original answer
    total.append(tmp)

    
cqas = pd.DataFrame(total)

elasticsearch:   0%|          | 0/600 [00:00<?, ?it/s]05/20/2021 03:47:07 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.014s]
05/20/2021 03:47:07 - INFO - elasticsearch -   POST http://localhost:9200/document/_search?size=30 [status:200 request:0.060s]
05/20/2021 03:47:07 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.030s]
05/20/2021 03:47:08 - INFO - elasticsearch -   POST http://localhost:9200/document/_search?size=30 [status:200 request:0.168s]
elasticsearch:   0%|          | 2/600 [00:00<01:23,  7.14it/s]05/20/2021 03:47:08 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.028s]
05/20/2021 03:47:08 - INFO - elasticsearch -   POST http://localhost:9200/document/_search?size=30 [status:200 request:0.126s]
elasticsearch:   0%|          | 3/600 [00:00<01:27,  6.85it/s]05/20/2021 03:47:08 - INFO - elasticsearch -   POST http://localhost:9200/document/_

NameError: name 'pd' is not defined

In [37]:
cqas.to_pickle("elastic.pkl")
# cqas = pd.read_pickle("elastic.pkl")